In [22]:
import pandas as pd
import os
import numpy as np
from math import cos, sin, atan2, sqrt, pi, radians, degrees, asin
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [23]:
def parse(x):
    x = x[:4] + ' ' + x[4:6] + ' '+x[6:]
    return datetime.strptime(x, '%Y %m %d')
def read_data(file):
    return pd.read_csv(file, parse_dates=['date'], date_parser=parse)

In [24]:
A = read_data('data1/A.csv')
B = read_data('data1/B.csv')
C = read_data('data1/C.csv')
D = read_data('data1/D.csv')
E = read_data('data1/E.csv')
n_features = A.shape[1] - 3

In [25]:
scaler = MinMaxScaler(feature_range=(0, 1))
n_step = 1
# n_features = 52
n_ob = n_step * n_features

In [26]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def process(cdf):
    v = []
    for i in range(cdf.region.max()+1):
        df = cdf[cdf.region == i].copy()
        df = df.set_index("date")
        df.drop(["city", "region"], axis=1, inplace=True)
        values = df.values
        values = values.astype('float32')
        # scaled = scaler.fit_transform(values)
        reframed = series_to_supervised(values, n_step, 1)
        # reframed.drop(reframed.columns[range(53, 104)], axis=1, inplace=True)
        v.append(reframed.values)
        a = v[0]
        for i in range(1, len(v)):
            a = np.concatenate((a, v[i]), axis=0)
    return a

In [27]:
# A1 = read_data('data/A1.csv')
a = process(A)
b = process(B)
c = process(C)
d = process(D)
e = process(E)
data = np.concatenate((a,b,c,d,e),axis = 0)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(data[:, :n_ob], data[:, n_ob:], test_size=0.2, random_state=2)

In [30]:
X_train = np.reshape(X_train, (X_train.shape[0], n_features, n_step))
X_test = np.reshape(X_test, (X_test.shape[0], n_features, n_step))
model = Sequential()
model.add(LSTM(units=128, input_shape=(X_test.shape[1], X_test.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(Dense(n_features))
# model.add(Dense(1))

model.compile(loss='msle', optimizer='adam')
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test), verbose=2)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 13798 samples, validate on 3450 samples
Epoch 1/10
 - 9s - loss: 0.5881 - val_loss: 0.3554
Epoch 2/10
 - 8s - loss: 0.3184 - val_loss: 0.2677
Epoch 3/10
 - 7s - loss: 0.2563 - val_loss: 0.2245
Epoch 4/10
 - 7s - loss: 0.2206 - val_loss: 0.1969
Epoch 5/10
 - 7s - loss: 0.1967 - val_loss: 0.1768
Epoch 6/10
 - 7s - loss: 0.1786 - val_loss: 0.1631
Epoch 7/10
 - 8s - loss: 0.1636 - val_loss: 0.1517
Epoch 8/10
 - 8s - loss: 0.1530 - val_loss: 0.1422
Epoch 9/10
 - 7s - loss: 0.1453 - val_loss: 0.1357
Epoch 10/10
 - 7s - loss: 0.1380 - val_loss: 0.1298


In [31]:
# Evaluation
yhat = model.predict(X_test)
# print(yhat)
# x_test0 = X_test.reshape((X_test.shape[0], n_features*n_step))
# inv_yhat = np.concatenate((yhat, x_test0[:, n_ob:]), axis=1)
# inv_yhat = scaler.inverse_transform(yhat)
# inv_yhat = inv_yhat[:, 0]
# y_test = y_test.reshape((len(y_test), n_features))
# inv_y = np.concatenate((y_test, x_test0[:, n_ob:]), axis=1)
# inv_y = scaler.inverse_transform(y_test)
# inv_y = inv_y[:, 0]

In [32]:
# RMSE
rmse = sqrt(mean_squared_error(y_test, yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 361.522


In [33]:
def process_last_day(cdf):
    v = []
    for i in range(cdf.region.max()+1):
        df = cdf[cdf.region == i].copy()
        df = df.set_index("date")
        df.drop(["city", "region"], axis=1, inplace=True)
        values = df.values
        values = values.astype('float32')
        # scaled = scaler.fit_transform(values)
        reframed = series_to_supervised(values, n_step, 1)
        v.append(np.array([reframed.values[-1]])[:,n_ob:])
    return v
        

In [34]:
a_lastday = process_last_day(A)
b_lastday = process_last_day(B)
c_lastday = process_last_day(C)
d_lastday = process_last_day(D)
e_lastday = process_last_day(E)

In [35]:
def predict(day_before):
    day_before = np.reshape(day_before, (day_before.shape[0], n_features, n_step))
    # day_before = np.reshape(day_before, (day_before.shape[0], n_step, n_features))
    day_after = model.predict(day_before)
    return day_after

In [36]:
def generate_future_prediction(lastday):
    res_city_lst = []
    for region_begin in lastday:
        res_date_lst = [predict(region_begin)]
        for i in range(1, 30):
            res_date_lst.append(predict(res_date_lst[-1]))
        res_city_lst.append(res_date_lst)
    return res_city_lst

In [37]:
a_prediction = generate_future_prediction(a_lastday)
b_prediction = generate_future_prediction(b_lastday)
c_prediction = generate_future_prediction(c_lastday)
d_prediction = generate_future_prediction(d_lastday)
e_prediction = generate_future_prediction(e_lastday)

In [38]:
a_prediction[0][0]

array([[37.663475  ,  0.29587582,  0.33411047,  7.479507  ]],
      dtype=float32)

In [39]:
def generate_res_infection_lst(cities_prediction):
    res_infection = []
    res_region = []
    for city_prediction in cities_prediction:
        region_id = 0
        for region in city_prediction:
            for date in region:
                res_infection.append(int(round(date[0][0])))
                res_region.append(region_id)
            region_id = region_id + 1
    return res_infection, res_region
def check_minus(infection):
    for i in range(len(infection)):
        if infection[i] < 0:
            if infection[i - 1] < 6:
                infection[i] = 0
            else:
                infection[i] = int(infection[i - 1])
    return infection

In [40]:
infection, region = generate_res_infection_lst([a_prediction,b_prediction,c_prediction,d_prediction,e_prediction])
infection = check_minus(infection)
submission = pd.read_csv('data/submission.csv', header=None, names=['city','region','date','infection'])
submission['infection'] = infection
submission['region'] = region
submission.to_csv('data/test_submission_double_LSTM.csv',index=False)